# Making HTTP Requests

In Python, there are many libraries to make HTTP requests. We will use a 3rd-party library called "requests", which is very easy to use. 

Making a "GET" request is as simple as: 

```python
import requests

res = requests.get(url) # returns a "Response" object
res.content # has the "body" of the response
```

You might need to install the requests library! 

You can do that with the following code in a Jupyter cell: 

```python
! pip install requests
```

Or, if you're using anaconda, optionally you can also do: 

```python
! conda install -c anaconda requests
```

## Parsing JSON data

To parse JSON data in Python, we will use the "json" module: 

```python
import json
```

Read more about the module on the [documentation page](https://docs.python.org/3/library/json.html)!

All we care about for this part is the method "loads", which turns JSON data into a Python object: 

```python
json.loads(my_string_encoded_json)
```

## Pokemon API

There is a simple, open API called "pokeapi" that allows us to make requests and see how to use APIs. Like everything, we first look at the documentation: 

https://pokeapi.co/docs/v2.html

In [4]:
# Let's see how to make a get request to the API: 
import requests
import json
import pandas as pd

res = requests.get('https://pokeapi.co/api/v2/pokemon')
res_dict = json.loads(res.content)
res_dict

{'count': 964,
 'next': 'https://pokeapi.co/api/v2/pokemon?offset=20&limit=20',
 'previous': None,
 'results': [{'name': 'bulbasaur',
   'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
  {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
  {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
  {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
  {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
  {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
  {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
  {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
  {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
  {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
  {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
  {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
  {'name': 'weedle', 'ur

In [8]:
# Challenge: 
# Create a Dataframe with all the Pokemon names and their URLs. 

def get_pokes(url):
    res = requests.get(url)
    res_dict = json.loads(res.content)
    if res_dict['next']==None:
        return (None, res_dict['results'])
    else:
        return (res_dict['next'],res_dict['results'])
 
    # Make the HTTP request to the given url. 
    # Parse the response as json
    # return the "next" and the "results" (as a 2-tuple!)
    # make sure to return a "falsey" value (such as None)
    # if there is not a "next!"
    
def catch_em_all(url):
    pokes = [] 
    # While loop! Like a for-loop, 
    # but goes on for an indetermined amount
    # of time:
    while url:
        url, results = get_pokes(url)
        pokes += results
    return pokes

# This data is most naturally represented as a list of dictionaries. 
# How can we create a dataframe from a list of dictionaries? 
# Try to find out on your own, from the internet!

# TODO: turn list_of_pokes into a dataframe.
def df_pokes(url):
    pokes = catch_em_all(url)
    pokes_df = pd.DataFrame(pokes)
    return pokes_df


## Project: Live Exchange Rates

Imagine that you work with financial assets which are denominated in different currencies. You analyze this data regularly, and want to create a "transformation" function that transforms all your assets into EUR prices, based on today's exchange rate. 

Your data with the local-currency-denominated value of each asset lives in a file called "assets.csv" which should be located in the same folder as this notebook. 

Write a "data loading" function that: 

1. Reads the data, given the path to the file. 
2. Returns a dataframe with an additional column that has the assets value in euros, as of today.

Use this free API to get today's exchange rates: https://exchangeratesapi.io/. You will need to read the documentation and try it out to see how it works. 

HINT: Write a separate function to get the current exchange rates! That can be reused!

In [14]:

import requests
import json

def get_exchange_rates(curr):
    res = requests.get('https://api.exchangeratesapi.io/latest?base={}'.format(curr))
    rates_full = json.loads(res.content)
    rates = rates_full['rates']
    rates_df = pd.DataFrame.from_dict(rates, orient='index', columns=['exchange_rate'])
    return rates_df

def get_data(path,curr='EUR'):
    rates_df = get_exchange_rates(curr)
    assets = pd.read_csv(path)
    assets_new = pd.merge(assets, rates_df, how = "left", left_on = 'curr', right_index=True)
    assets_new['value_{}'.format(curr)] = assets_new['value']/assets_new['exchange_rate']
    assets_new.drop(columns='exchange_rate',inplace=True)
    return assets_new
    
assets_EUR = get_data('assets.csv')

assets_EUR

,value,curr,value_EUR
0,48.910052,THB,1.451811
1,16.505115,THB,0.489926
2,30.370579,INR,0.384825
3,14.126967,SEK,1.319846
4,23.406904,HKD,2.676268
...,...,...,...
995,13.593894,HRK,1.824978
996,41.710860,ZAR,2.523404
997,12.877760,AUD,7.969898
998,29.561696,KRW,0.022794
